# Jupiter Notebook of all the parsings

In [3]:
%pip install faiss-cpu
%pip install ijson

from universal_parser import *

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Mount the drive on collab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## MediaSum

### Download

In [ ]:
!wget https://huggingface.co/datasets/ccdv/mediasum/resolve/main/train_data.zip
!unzip train_data.zip

### Process Dataset

In [ ]:
parser = MediaSumParser()

parser.convert(
    dataset_path    = "./train_data.txt",
    output_path     = "/content/drive/MyDrive/media_sum.csv",
    to_process      = 463596
)

## SODA

### Download

In [ ]:
!wget https://huggingface.co/datasets/allenai/soda/resolve/main/train.parquet

### Process Dataset

In [4]:
parser = SODA()

parser.convert(
    dataset_path    = "./train.parquet",
    output_path     = "/content/drive/MyDrive/soda.csv",
    read_format     = "parquet"
)

To_process: 1191582


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/soda.csv'

## SamSUM

### Download

In [ ]:
!wget https://huggingface.co/datasets/knkarthick/samsum/resolve/main/train.csv

### Process Dataset

In [ ]:
parser = SamSUM()

parser.convert(
    dataset_path    = "./train.csv",
    output_path     = "/content/drive/MyDrive/samsum.csv",
    read_format     = "csv"
)

## CSVs to Parquet 

In [1]:
%pip install datasets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
datasets_paths = [
    "datasets/MediaSum/parsed.csv",
    "datasets/SamSUM/parsed.csv",
    "datasets/SODA/parsed.csv"
]

In [6]:
import os
import pandas as pd
from datasets import Dataset, concatenate_datasets
import pyarrow as pa

parquet_output_dir = "sharded_parquet"
batch_size = 50000
os.makedirs(parquet_output_dir, exist_ok=True)

def write_chunk_to_parquet(chunk_df, shard_idx):
    dataset = Dataset.from_pandas(chunk_df, preserve_index=False)
    dataset.to_parquet(f"{parquet_output_dir}/shard_{shard_idx:05d}.parquet")

shard_idx = 0
for csv_file in datasets_paths:

    print(f"Processing: {csv_file}")
    csv_path = os.path.join(csv_file)
    
    for chunk in pd.read_csv(csv_path, chunksize=batch_size):
        write_chunk_to_parquet(chunk, shard_idx)
        shard_idx += 1


Processing: datasets/MediaSum/parsed.csv


Creating parquet from Arrow format: 100%|██████████| 50/50 [00:02<00:00, 18.75ba/s]


Processing: datasets/SamSUM/parsed.csv


Creating parquet from Arrow format: 100%|██████████| 15/15 [00:00<00:00, 233.48ba/s]


Processing: datasets/SODA/parsed.csv


Creating parquet from Arrow format: 100%|██████████| 42/42 [00:00<00:00, 192.75ba/s]


## Push to Huggingface

In [9]:
%pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 574.3 kB/s eta 0:00:001m563.6 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from datasets import load_dataset

streamable_dataset = load_dataset("parquet", data_files=f"{parquet_output_dir}/*.parquet", split="train")
streamable_dataset.push_to_hub("JustinDuc/MultiDomain-QADialog", token="[YOUR TOKEN]")


pushing


Uploading the dataset shards: 100%|██████████| 11/11 [17:18<00:00, 94.43s/it]
Repo card metadata block was not found. Setting CardData to empty.


pushed
